In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from sklearn.metrics import mean_absolute_error

In [2]:
train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
test  = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
item = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")


In [3]:
def basic_info(data):
    print("*************** TOP 10 RECOED ***************")
    print(data.head(10))
    print("*************** INFO ***************")
    print(data.info())
    print("*************** DESCRIBE ***************")
    print(data.describe())
    print("*************** COLUMNS ***************")
    print(data.columns)
    print("*************** MISSING VALUES ***************")
    print(data.isnull().sum())
    print("*************** SHAPE ***************")
    print(data.shape)

print("=============== TRAIN ===============")
basic_info(train)
print("=============== TEST ===============")
basic_info(test)
print("=============== ITEM CAT ===============")
basic_info(item_categories)
print("=============== SHOPS ===============")
basic_info(shops)
print("=============== ITEMS ===============")
basic_info(item)

=============== TRAIN ===============
*************** TOP 10 RECOED ***************
         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
5  10.01.2013               0       25     2564      349.00           1.0
6  02.01.2013               0       25     2565      549.00           1.0
7  04.01.2013               0       25     2572      239.00           1.0
8  11.01.2013               0       25     2572      299.00           1.0
9  03.01.2013               0       25     2573      299.00           3.0
*************** INFO ***************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries

In [4]:
train['date'] = pd.to_datetime(train['date'], format = "%d.%m.%Y" )
pt_train = pd.pivot_table(train, index = ['shop_id', 'item_id'], values = 'item_cnt_day', columns = ['date_block_num'], aggfunc = np.sum, fill_value = 0)
pt_train.reset_index(inplace = True)


pt_test = pd.merge(test, pt_train, on=['shop_id', 'item_id'], how = 'left')
pt_test.fillna(0, inplace=True)

X = pt_train.drop(columns=['shop_id','item_id', 33], axis=1)
y = pt_train[33]

pt_test.drop(columns=['shop_id','item_id', 'ID', 0], axis=1, inplace = True)
pt_test.columns = X.columns

X_train = np.expand_dims(X, axis =2)
y_train = np.expand_dims(y, axis =1)

X_test = np.expand_dims(pt_test, axis =2)

model = Sequential()
model.add(LSTM(128, activation = "relu", input_shape =(X_train.shape[1], X_train.shape[2]), return_sequences = True))
model.add(LSTM(64, activation = "relu", return_sequences = False))
model.add(Dropout(0.1))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mean_squared_error'])

hist = model.fit(X_train, y_train, epochs= 10, batch_size = 1024, verbose = 1, shuffle = True, validation_split = 0.3)
pred = model.predict(X_test)

output = pd.DataFrame({'ID': test["ID"], 'item_cnt_month': pred.flatten()})
output.to_csv('submission.csv', index=False)

2022-06-29 11:31:10.930020: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-06-29 11:31:11.381070: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
290/290 [==============================] - 163s 551ms/step - loss: 25086.1543 - mean_squared_error: 25086.1543 - val_loss: 122.0547 - val_mean_squared_error: 122.0547
Epoch 2/10
290/290 [==============================] - 157s 542ms/step - loss: 443.5835 - mean_squared_error: 443.5835 - val_loss: 1.4269 - val_mean_squared_error: 1.4269
Epoch 3/10
290/290 [==============================] - 156s 539ms/step - loss: 71.8194 - mean_squared_error: 71.8194 - val_loss: 164.1698 - val_mean_squared_error: 164.1698
Epoch 4/10
290/290 [==============================] - 156s 539ms/step - loss: 48.0463 - mean_squared_error: 48.0463 - val_loss: 7.3167 - val_mean_squared_error: 7.3167
Epoch 5/10
290/290 [==============================] - 156s 540ms/step - loss: 28.7622 - mean_squared_error: 28.7622 - val_loss: 5.5938 - val_mean_squared_error: 5.5938
Epoch 6/10
290/290 [==============================] - 158s 545ms/step - loss: 30.1521 - mean_squared_error: 30.1521 - val_loss: 1.3302 - val_mea